## 1. Importar Bibliotecas

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from src.models.train import ModelTrainer
from src.models.export import export_sklearn_to_onnx, export_model_joblib

%matplotlib inline

## 2. Carregar Dados Processados

In [ ]:
# Carregar dados processados
processed_dir = Path('../data/processed')

X_train = np.load(processed_dir / 'X_train.npy')
X_test = np.load(processed_dir / 'X_test.npy')
y_train = np.load(processed_dir / 'y_train.npy')
y_test = np.load(processed_dir / 'y_test.npy')

print(f"Train set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

## 3. Inicializar ModelTrainer

In [ ]:
trainer = ModelTrainer(models_dir='../models')
trainer.initialize_models()

print("Modelos disponíveis:")
for model_name in trainer.models.keys():
    print(f"  - {model_name}")

## 4. Treinar Todos os Modelos

In [ ]:
# Treinar todos os modelos
trainer.train_all_models(X_train, y_train, X_test, y_test)

## 5. Comparar Resultados

In [ ]:
# Criar DataFrame com resultados
results_df = pd.DataFrame(trainer.results).T
results_df = results_df.round(4)
print("\nResultados dos Modelos:")
results_df

In [ ]:
# Visualizar comparação
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
results_df[['train_accuracy', 'test_accuracy']].plot(kind='bar', ax=axes[0])
axes[0].set_title('Accuracy - Train vs Test')
axes[0].set_ylabel('Accuracy')
axes[0].set_xlabel('Model')
axes[0].legend(['Train', 'Test'])
axes[0].set_ylim([0, 1.1])

# Métricas de Test
results_df[['test_precision', 'test_recall', 'test_f1']].plot(kind='bar', ax=axes[1])
axes[1].set_title('Test Metrics Comparison')
axes[1].set_ylabel('Score')
axes[1].set_xlabel('Model')
axes[1].legend(['Precision', 'Recall', 'F1-Score'])
axes[1].set_ylim([0, 1.1])

plt.tight_layout()
plt.show()

## 6. Otimização de Hiperparâmetros (Opcional)

In [ ]:
# Exemplo: Tuning para Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Descomentar para executar (pode demorar)
# best_params, best_score = trainer.hyperparameter_tuning(
#     'random_forest', 
#     param_grid, 
#     X_train, 
#     y_train, 
#     cv=3
# )

## 7. Salvar Modelos

In [ ]:
# Salvar todos os modelos treinados
for model_name in trainer.models.keys():
    trainer.save_model(model_name)

# Salvar resultados
trainer.save_results()

## 8. Exportar Melhor Modelo para ONNX

In [ ]:
# Obter melhor modelo
best_model_name, best_model = trainer.best_model
print(f"Melhor modelo: {best_model_name}")

# Exportar para ONNX
X_sample = pd.DataFrame(X_train[:5])  # Amostra para inferência de forma

try:
    onnx_path = export_sklearn_to_onnx(
        best_model,
        X_sample,
        output_path='../models',
        model_name='best_model'
    )
    print(f"\nModelo exportado para: {onnx_path}")
except Exception as e:
    print(f"Erro ao exportar para ONNX: {e}")
    print("Salvando com joblib como alternativa...")
    export_model_joblib(best_model, '../models', 'best_model')

## 9. Testar Modelo ONNX

In [ ]:
# Testar modelo ONNX
try:
    import onnxruntime as rt
    
    # Carregar modelo ONNX
    onnx_path = '../models/best_model.onnx'
    sess = rt.InferenceSession(onnx_path)
    
    # Fazer predições
    input_name = sess.get_inputs()[0].name
    predictions = sess.run(None, {input_name: X_test[:5].astype(np.float32)})[0]
    
    print("Predições ONNX (primeiras 5):")
    print(predictions)
    print(f"\nLabels reais (primeiras 5):")
    print(y_test[:5])
    
except Exception as e:
    print(f"Erro ao testar modelo ONNX: {e}")

## 10. Conclusões

TODO: Adicionar conclusões:
- Melhor modelo e justificativa
- Comparação de desempenho
- Possíveis melhorias
- Próximos passos